In [ ]:
!pip3 install mtcnn

     |████████████████████████████████| 2.3MB 6.7MB/s 


In [ ]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.models import load_model 
from keras import backend as K
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense,Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from sklearn import metrics
import math
import numpy as np
from sklearn.metrics import confusion_matrix
import math
import numpy as np
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import cv2
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def face(fil):
  final_size=(160, 160)
  img = Image.open(fil)
  img = img.convert('RGB')
  arr = asarray(img)
  ret = MTCNN().detect_faces(arr)
  x1, y1, width, height = ret[0]['box']

  if int(height)==0 and int(width)==0:
    return arr

  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height
  extracted_face = arr[y1:y2, x1:x2]
  img = Image.fromarray(extracted_face)
  img = img.resize(final_size)
  ans = asarray(img)
  return ans

In [ ]:
def make_X(dir):
  i=1
  X = list()
  path=glob.glob(dir)
  for src in path:
    X.append(face(src))
    print(i)
    i+=1
  return X

In [ ]:
def load(dir):
  X=list()
  y=list()
  for subdr in listdir(dir):
    path = dir + subdr +"/*jpg"
    f = make_X(path)
    label = [subdr for _ in range(len(f))]
    X.extend(f)
    y.extend(label)
  return asarray(X), asarray(y)

In [ ]:
Xtrain, ytrain = load('/content/drive/MyDrive/data/train/')
np.savez_compressed('/content/drive/MyDrive/face_data_train.npz',  Xtrain, ytrain)

1 (160, 160, 3)
2 (160, 160, 3)
3 (160, 160, 3)
4 (160, 160, 3)
5 (160, 160, 3)
6 (160, 160, 3)
7 (160, 160, 3)
8 (160, 160, 3)
9 (160, 160, 3)
10 (160, 160, 3)
11 (160, 160, 3)
12 (160, 160, 3)
13 (160, 160, 3)
14 (160, 160, 3)
15 (160, 160, 3)
16 (160, 160, 3)
17 (160, 160, 3)
18 (160, 160, 3)
19 (160, 160, 3)
20 (160, 160, 3)
21 (160, 160, 3)
22 (160, 160, 3)
23 (160, 160, 3)
24 (160, 160, 3)
25 (160, 160, 3)
26 (160, 160, 3)
27 (160, 160, 3)
28 (160, 160, 3)
29 (160, 160, 3)
30 (160, 160, 3)
31 (160, 160, 3)
32 (160, 160, 3)
33 (160, 160, 3)
34 (160, 160, 3)
35 (160, 160, 3)
36 (160, 160, 3)
37 (160, 160, 3)
38 (160, 160, 3)
39 (160, 160, 3)
40 (160, 160, 3)
41 (160, 160, 3)
42 (160, 160, 3)
43 (160, 160, 3)
44 (160, 160, 3)
45 (160, 160, 3)
46 (160, 160, 3)
47 (160, 160, 3)
48 (160, 160, 3)
49 (160, 160, 3)
50 (160, 160, 3)
51 (160, 160, 3)
52 (160, 160, 3)
53 (160, 160, 3)
54 (160, 160, 3)
55 (160, 160, 3)
56 (160, 160, 3)
57 (160, 160, 3)
58 (160, 160, 3)
59 (160, 160, 3)
60 (16

In [ ]:
data = np.load('/content/drive/MyDrive/face_data_train.npz')
Xtrain, ytrain= data['arr_0'], data['arr_1']

Dataset: train=3491
Loaded:  (3491, 160, 160, 3) (3491,)


In [ ]:
model = load_model('/content/drive/MyDrive/facenet_keras.h5')
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [ ]:
def embed(model, f):
	f = f.astype('float32')
	m, s = f.mean(), f.std()
	f = (f - m) / s
	samples = np.expand_dims(f, axis=0)
	y = model.predict(samples)
	return y[0]

In [ ]:
nXTrain = list()
for face in trainX:
	embeddings = embed(model, face)
	nXTrain.append(embeddings)
nXTrain = np.asarray(nXTrain)
np.savez_compressed('/content/drive/MyDrive/face_train_embed.npz',nXTrain, ytrain)

Cause: could not parse the source code of <function <lambda> at 0x7f75ba853cb0>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7f75ba853cb0>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7f75ba8538c0>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7f75ba8538c0>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7f75ba853a70>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could no

In [ ]:
data = np.load('/content/drive/MyDrive/face spoof new/face_data_test.npz')
testX, testy= data['arr_0'], data['arr_1']
print('Dataset: train=%d' % (testX.shape[0]))
print(testy)

Dataset: train=3491
['spoof' 'spoof' 'spoof' ... 'original' 'original' 'original']
